In [ ]:
import numpy as np
import random
import pandas as pd
import joblib
import psutil

In [ ]:
from collections import defaultdict
import lightgbm as lgb
from matplotlib import pyplot as plt
import riiideducation
from sklearn.metrics import roc_auc_score
import gc

_ = np.seterr(divide='ignore', invalid='ignore')

# Preprocess

In [ ]:
data_types_dict = {
    'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    'content_type_id':'int8', 
    'task_container_id': 'int16',
    #'user_answer': 'int8',
    'answered_correctly': 'int8', 
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'bool'
}
target = 'answered_correctly'

In [ ]:
questions_df = pd.read_pickle('../input/train-df2/questions_df.pkl')
attempt_no_agg = pd.read_pickle('../input/train-df2/attempt_no_agg.pkl')
explanation_agg = pd.read_pickle('../input/train-df2/explanation_agg.pkl')
max_timestamp_u = pd.read_pickle('../input/train-df2/max_timestamp_u.pkl')
max_timestamp_u2 = pd.read_pickle('../input/train-df2/max_timestamp_u2.pkl')
max_timestamp_u3 = pd.read_pickle('../input/train-df2/max_timestamp_u3.pkl')
task_container_agg = pd.read_pickle('../input/train-df2/task_container_agg.pkl')
user_agg = pd.read_pickle('../input/train-df2/user_agg.pkl')
user_lecture_agg = pd.read_pickle('../input/train-df2/user_lecture_agg.pkl')
user_prior_question_elapsed_time = pd.read_pickle('../input/train-df2/user_prior_question_elapsed_time.pkl')

print(psutil.virtual_memory().percent)

In [ ]:
features_dict = {
    'timestamp':'float16',#
    'user_interaction_count':'int16',
    'user_interaction_timestamp_mean':'float32',
    'lagtime':'float32',#
    'lagtime2':'float32',
    'lagtime3':'float32',
    'content_id':'int16',
    'task_container_id':'int16',
    'user_lecture_sum':'int16',#
    'user_lecture_lv':'float16',##
    'prior_question_elapsed_time':'float32',#
    'delta_prior_question_elapsed_time':'int32',#
    'user_correctness':'float16',#
    'user_uncorrect_count':'int16',#
    'user_correct_count':'int16',#
    'content_correctness_std':'float16',
    'content_correct_count':'int32',
    'content_uncorrect_count':'int32',#
    'content_elapsed_time_mean':'float16',
    'content_had_explanation_mean':'float16',
    'content_explation_false_mean':'float16',
    'content_explation_true_mean':'float16',
    'task_container_correctness':'float16',
    'task_container_std':'float16',
    'task_container_cor_count':'int32',#
    'task_container_uncor_count':'int32',#
    'attempt_no':'int8',#
    'part':'int8',
    'part_correctness_mean':'float16',
    'part_correctness_std':'float16',
    'part_uncor_count':'int32',
    'part_cor_count':'int32',
    'tags0': 'int8',
    'tags1': 'int8',
    'tags2': 'int8',
    'tags3': 'int8',
    'tags4': 'int8',
    'tags5': 'int8',
    'part_bundle_id':'int32',
    'content_sub_bundle':'int8',
    'prior_question_had_explanation':'int8',
    'explanation_mean':'float16',
    'explanation_false_count':'int16',
    'explanation_true_count':'int16',
}
categorical_columns= [
    'content_id',
    'task_container_id',
    'part',
    'tags0',
    'tags1',
    'tags2',
    'tags3',
    'tags4',
    'tags5',
    'part_bundle_id',
    'content_sub_bundle',
    'prior_question_had_explanation', 
]

features=list(features_dict.keys())


## Load NN-SAKT model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


MAX_SEQ = 100

class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=MAX_SEQ, embed_dim=128):
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)

        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        device = x.device        
        x = self.embedding(x)
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)

        pos_x = self.pos_embedding(pos_id)
        x = x + pos_x

        e = self.e_embedding(question_ids)

        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1), att_weight
    
    
skills = joblib.load("/kaggle/input/riiid-sakt-model-dataset-public/skills.pkl.zip")
n_skill = len(skills)
group = joblib.load("/kaggle/input/riiid-sakt-model-dataset-public/group.pkl.zip")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

nn_model = SAKTModel(n_skill, embed_dim=128)
try:
    nn_model.load_state_dict(torch.load("/kaggle/input/riiid-sakt-model-dataset-public/sakt_model.pt"))
except:
    nn_model.load_state_dict(torch.load("/kaggle/input/riiid-sakt-model-dataset-public/sakt_model.pt", map_location='cpu'))
nn_model.to(device)
nn_model.eval()

# Inference

In [ ]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, skills, max_seq=MAX_SEQ): 
        super(TestDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.skills = skills
        self.n_skill = len(skills)
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)

        if user_id in self.samples.index:
            q_, qa_ = self.samples[user_id]
            
            seq_len = len(q_)

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_          
        
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[1:].copy()
        x += (qa[1:] == 1) * self.n_skill
        
        questions = np.append(q[2:], [target_id])
        
        return x, questions

In [ ]:
user_sum_dict = user_agg['sum'].astype('int16').to_dict(defaultdict(int))
user_count_dict = user_agg['count'].astype('int16').to_dict(defaultdict(int))

In [ ]:
del user_agg
gc.collect()

task_container_sum_dict = task_container_agg['sum'].astype('int32').to_dict(defaultdict(int))
task_container_count_dict = task_container_agg['count'].astype('int32').to_dict(defaultdict(int))
task_container_std_dict = task_container_agg['var'].astype('float16').to_dict(defaultdict(int))

explanation_sum_dict = explanation_agg['sum'].astype('int16').to_dict(defaultdict(int))
explanation_count_dict = explanation_agg['count'].astype('int16').to_dict(defaultdict(int))
del task_container_agg
del explanation_agg
gc.collect()

In [ ]:
user_lecture_sum_dict = user_lecture_agg['sum'].astype('int16').to_dict(defaultdict(int))
user_lecture_count_dict = user_lecture_agg['count'].astype('int16').to_dict(defaultdict(int))

del user_lecture_agg
gc.collect()

In [ ]:
max_timestamp_u_dict=max_timestamp_u.set_index('user_id').to_dict()
max_timestamp_u_dict2=max_timestamp_u2.set_index('user_id').to_dict()
max_timestamp_u_dict3=max_timestamp_u3.set_index('user_id').to_dict()
user_prior_question_elapsed_time_dict=user_prior_question_elapsed_time.set_index('user_id').to_dict()
#del question_elapsed_time_agg
del max_timestamp_u
del max_timestamp_u2
del max_timestamp_u3
del user_prior_question_elapsed_time
gc.collect()

In [ ]:
attempt_no_sum_dict = attempt_no_agg['sum'].to_dict(defaultdict(int))

del attempt_no_agg
gc.collect()

In [ ]:
def get_max_attempt(user_id,content_id):
    k = (user_id,content_id)

    if k in attempt_no_sum_dict.keys():
        attempt_no_sum_dict[k]+=1
        return attempt_no_sum_dict[k]

    attempt_no_sum_dict[k] = 1
    return attempt_no_sum_dict[k]

In [ ]:
model = lgb.Booster(model_file='../input/riiid-lgbm-1500/model_1500.txt')

In [ ]:
env = riiideducation.make_env()

In [ ]:
iter_test = env.iter_test()
prior_test_df = None
prev_test_df1 = None

In [ ]:
# from training

lagtime_mean = 20441941.566456214
lagtime_mean2 = 40370652.30368404
lagtime_mean3 = 60008399.99610778
delta_prior_question_elapsed_time_mean = 44.682064056396484
prior_question_elapsed_time_mean = 13005.0810546875

In [ ]:
%%time

for (test_df, sample_prediction_df) in iter_test:
    
    test_df1=test_df.copy()
    if (prev_test_df1 is not None):
        prev_test_df1['answered_correctly'] = eval(test_df1['prior_group_answers_correct'].iloc[0])
        prev_test_df1 = prev_test_df1[prev_test_df1.content_type_id == False]
        
        prev_group = prev_test_df1[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))
        for prev_user_id in prev_group.index:
            if prev_user_id in group.index:
                group[prev_user_id] = (
                    np.append(group[prev_user_id][0], prev_group[prev_user_id][0])[-MAX_SEQ:], 
                    np.append(group[prev_user_id][1], prev_group[prev_user_id][1])[-MAX_SEQ:]
                )
 
            else:
                group[prev_user_id] = (
                    prev_group[prev_user_id][0], 
                    prev_group[prev_user_id][1]
                )

    prev_test_df1 = test_df1.copy()
    
    test_df1 = test_df1[test_df1.content_type_id == False]
    test_dataset = TestDataset(group, test_df1, skills)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, shuffle=False)
    
    outs = []

    for item in test_dataloader:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        with torch.no_grad():
            output, att_weight = nn_model(x, target_id)
        outs.extend(torch.sigmoid(output)[:, -1].view(-1).data.cpu().numpy())
    
    ###for lgb
    if prior_test_df is not None:
        prior_test_df[target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df['prior_question_had_explanation'].fillna(False, inplace=True)       
        prior_test_df.prior_question_had_explanation=prior_test_df.prior_question_had_explanation.astype('int8')
    
        user_ids = prior_test_df['user_id'].values
        targets = prior_test_df[target].values        
        
        for user_id, answered_correctly in zip(user_ids,targets):
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1


    prior_test_df = test_df.copy() 
        
    
    question_len=len( test_df[test_df['content_type_id'] == 0])
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df.prior_question_had_explanation=test_df.prior_question_had_explanation.astype('int8')
    
    test_df['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean, inplace=True)
    

    user_lecture_sum = np.zeros(question_len, dtype=np.int16)
    user_lecture_count = np.zeros(question_len, dtype=np.int16) 
    
    user_sum = np.zeros(question_len, dtype=np.int16)
    user_count = np.zeros(question_len, dtype=np.int16)

    task_container_sum = np.zeros(question_len, dtype=np.int32)
    task_container_count = np.zeros(question_len, dtype=np.int32)
    task_container_std = np.zeros(question_len, dtype=np.float16)

    explanation_sum = np.zeros(question_len, dtype=np.int32)
    explanation_count = np.zeros(question_len, dtype=np.int32)
    delta_prior_question_elapsed_time = np.zeros(question_len, dtype=np.int32)

    attempt_no_count = np.zeros(question_len, dtype=np.int16)
    lagtime = np.zeros(question_len, dtype=np.float32)
    lagtime2 = np.zeros(question_len, dtype=np.float32)
    lagtime3 = np.zeros(question_len, dtype=np.float32)
   
    i=0
    for j, (user_id,prior_question_had_explanation,content_type_id,prior_question_elapsed_time,timestamp, content_id,task_container_id) in enumerate(zip(test_df['user_id'].values,test_df['prior_question_had_explanation'].values,test_df['content_type_id'].values,test_df['prior_question_elapsed_time'].values,test_df['timestamp'].values, test_df['content_id'].values, test_df['task_container_id'].values)):
        
         #
        user_lecture_sum_dict[user_id] += content_type_id
        user_lecture_count_dict[user_id] += 1
        if(content_type_id==1):#
            x=1
        if(content_type_id==0):#   
            user_lecture_sum[i] = user_lecture_sum_dict[user_id]
            user_lecture_count[i] = user_lecture_count_dict[user_id]
                
            user_sum[i] = user_sum_dict[user_id]
            user_count[i] = user_count_dict[user_id]

            task_container_sum[i] = task_container_sum_dict[task_container_id]
            task_container_count[i] = task_container_count_dict[task_container_id]
            task_container_std[i]=task_container_std_dict[task_container_id]

            explanation_sum_dict[user_id] += prior_question_had_explanation
            explanation_count_dict[user_id] += 1
            explanation_sum[i] = explanation_sum_dict[user_id]
            explanation_count[i] = explanation_count_dict[user_id]

            if user_id in max_timestamp_u_dict['max_time_stamp'].keys():
                lagtime[i]=timestamp-max_timestamp_u_dict['max_time_stamp'][user_id]
                if(max_timestamp_u_dict2['max_time_stamp2'][user_id]==lagtime_mean2):#
                    lagtime2[i]=lagtime_mean2
                    lagtime3[i]=lagtime_mean3
                else:
                    lagtime2[i]=timestamp-max_timestamp_u_dict2['max_time_stamp2'][user_id]
                    if(max_timestamp_u_dict3['max_time_stamp3'][user_id]==lagtime_mean3):
                        lagtime3[i]=lagtime_mean3 #
                    else:
                        lagtime3[i]=timestamp-max_timestamp_u_dict3['max_time_stamp3'][user_id]
                    
                    max_timestamp_u_dict3['max_time_stamp3'][user_id]=max_timestamp_u_dict2['max_time_stamp2'][user_id]
                        
                max_timestamp_u_dict2['max_time_stamp2'][user_id]=max_timestamp_u_dict['max_time_stamp'][user_id]
                max_timestamp_u_dict['max_time_stamp'][user_id]=timestamp
            else:
                lagtime[i]=lagtime_mean
                max_timestamp_u_dict['max_time_stamp'].update({user_id:timestamp})
                lagtime2[i]=lagtime_mean2#
                max_timestamp_u_dict2['max_time_stamp2'].update({user_id:lagtime_mean2})
                lagtime3[i]=lagtime_mean3#
                max_timestamp_u_dict3['max_time_stamp3'].update({user_id:lagtime_mean3})

            if user_id in user_prior_question_elapsed_time_dict['prior_question_elapsed_time'].keys():            
                delta_prior_question_elapsed_time[i]=prior_question_elapsed_time-user_prior_question_elapsed_time_dict['prior_question_elapsed_time'][user_id]
                user_prior_question_elapsed_time_dict['prior_question_elapsed_time'][user_id]=prior_question_elapsed_time
            else:           
                delta_prior_question_elapsed_time[i]=delta_prior_question_elapsed_time_mean    
                user_prior_question_elapsed_time_dict['prior_question_elapsed_time'].update({user_id:prior_question_elapsed_time})
            i=i+1 
        
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    test_df=test_df.merge(questions_df.loc[questions_df.index.isin(test_df['content_id'])],
                  how='left', on='content_id', right_index=True)
    
    test_df['user_lecture_lv'] = user_lecture_sum / user_lecture_count
    test_df['user_lecture_sum'] = user_lecture_sum
    
    test_df['user_interaction_count'] = user_lecture_count
    test_df['user_interaction_timestamp_mean'] = test_df['timestamp']/user_lecture_count
    
    test_df['user_correctness'] = user_sum / user_count
    test_df['user_uncorrect_count'] =user_count-user_sum
    test_df['user_correct_count'] =user_sum
    
    test_df['task_container_correctness'] = task_container_sum / task_container_count
    test_df['task_container_cor_count'] = task_container_sum 
    test_df['task_container_uncor_count'] =task_container_count-task_container_sum 
    test_df['task_container_std'] = task_container_std 
    
    test_df['explanation_mean'] = explanation_sum / explanation_count
    test_df['explanation_true_count'] = explanation_sum
    test_df['explanation_false_count'] = explanation_count-explanation_sum 
    
    test_df['delta_prior_question_elapsed_time'] = delta_prior_question_elapsed_time 
    
    test_df["attempt_no"] = test_df[["user_id", "content_id"]].apply(lambda row: get_max_attempt(row["user_id"], row["content_id"]), axis=1)
    test_df["lagtime"]=lagtime
    test_df["lagtime2"]=lagtime2
    test_df["lagtime3"]=lagtime3

    test_df['timestamp']=test_df['timestamp']/(1000*3600)
    test_df.timestamp=test_df.timestamp.astype('float16')
    test_df['lagtime']=test_df['lagtime']/(1000*3600)
    test_df.lagtime=test_df.lagtime.astype('float32')
    test_df['lagtime2']=test_df['lagtime2']/(1000*3600)
    test_df.lagtime2=test_df.lagtime2.astype('float32')
    test_df['lagtime3']=test_df['lagtime3']/(1000*3600)
    test_df.lagtime3=test_df.lagtime3.astype('float32')
    test_df['user_interaction_timestamp_mean']=test_df['user_interaction_timestamp_mean']/(1000*3600)
    test_df.user_interaction_timestamp_mean=test_df.user_interaction_timestamp_mean.astype('float32')
    
    test_df['user_correctness'].fillna(0.67, inplace=True)

    sub_preds = np.zeros(test_df.shape[0])
    test_preds = model.predict(test_df[features])
    sub_preds += test_preds
    
    test_df[target]=0.35*np.array(outs)+0.65*np.array(sub_preds)
    
    env.predict(test_df[['row_id', target]])
    